In [2]:
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn import metrics

# Extraction des caractéristiques

In [ ]:
def extract_features(file_name, **args):
    """
   Extraction des caractéristiques d'un fichier audio .wav :
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        Usage :
        features = extract_features(file, mel=True, mfcc=True)
    """
    mfcc = args.get("mfcc")
    chroma = args.get("chroma")
    mel = args.get("mel")
    contrast = args.get("contrast")
    tonnetz = args.get("tonnetz")
    
    """
    Chargement d'un fichier audio
        X = Amplitude du son
        sample_rate -> Fréquence de l'audio
        sr -> La fréquence audio souhaité (None = Fréquence native de l’audio) 
        mono -> Si on veut un audio encodé en mono
    """
    X, sample_rate = librosa.load(file_name, sr=16000, mono=True) 
        
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        # mfccs recoit plusieur moyenne de mfcc qui sont transposé en colonne
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        #hstack permet de concaténer deux tableaux à la suite en ligne 
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    return result

# Chargement des datasets utilisés

In [ ]:
# Pour chaque dataset on a dictionnaire permettant d'obtenir l'émotion à partir du code utilisé :

ravdessToEmotion = {
    "01": "neutral",
    "02": "calm", # Spécifique à ce dataset
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

emodbToEmotion = {
    "N": "neutral",   
    "F": "happy",
    "T": "sad",
    "W": "angry",
    "A": "fearful",
    "E": "disgust",
    "L": "boredom" # Spécifique à ce dataset
}

tessToEmotion = {
    "neutral": "neutral",   
    "happy": "happy",
    "sad": "sad",
    "angry": "angry",
    "fear": "fearful",
    "disgust": "disgust",
    "ps": "surprised" # En réalité "pleasent surprise"=agréable surprise
}

saveeToEmotion = {
    'n': "neutral",
    'h': "happy",
    'sa': "sad",
    'a': "angry", 
    'f': "fearful",
    'd': "disgust",
    'su': "surprised"
}

cremadToEmotion = {
    "NEU": "neutral",
    "HAP": "happy",
    "SAD": "sad",
    "ANG":"angry",
    "FEA": "fearful",
    "DIS": "disgust",
    
}

SELECTION_EMOTIONS = { # Dictionnaire des émotions auxquelles ont s'intéresse
    "neutral",
    "sad",
    "happy",
    "angry"
}

SELECTION_DATASETS = { # Dictionnaire des datasets à utiliser
    #"cremad",
    #"emodb",
    "ravdess",
    "tess",
    "savee"
}

In [ ]:
import re

# Fonctions utilisées pour connaître l'émotion correspondante au fichier, selon le dataset:
def ravdessConverter(name):
    return ravdessToEmotion[name.split("-")[2]]
            
def emodbConverter(name):
    return emodbToEmotion[name[5]]

def cremadConverter(name):
    return cremadToEmotion[name.split("_")[2]]

def tessConverter(name):
    return tessToEmotion[name.split("_")[2].split(".")[0]]

def saveeConverter(name):
    return saveeToEmotion[re.sub('[0-9]|\.wav','', name)]
            

def fileToEmotion(dataset, file):
    """
    Extraction pour un dataset donné et un fichier donné de l'émotion correspondante
    """
    switcher = { # Dictionnaire des fonctions à utiliser
        "ravdess": ravdessConverter,
        "cremad": cremadConverter,
        "emodb": emodbConverter,
        "tess": tessConverter,
        "savee": saveeConverter
    }
    func = switcher.get(dataset, lambda: "Dataset invalide")
    return func(file)
       
 

In [ ]:
def load_data(test_size=0.25):    
    """
    Chargement des données dans 2 listes :
        - X contient les caractéristiques/features
        - y contient l'émotion correspondante (pour les features au même index dans X)
    Puis séparation en 4 ensembles:
        - X_train et y_train pour l'entraînement d'un modèle
        - X_test et y_test pour le test d'un modèle
    """
    X, y = [], []
               
    for file in glob.glob("../data/**/*.wav", recursive=True): # Parcours de l'arborescence des datasets
        
        basename = os.path.basename(file) # Obtention du nom du fichier
        dataset = file.split("\\")[1][5:]
        
        if dataset not in SELECTION_DATASETS: # On regarde si on utilise ce dataset
            continue
            
        emotion = fileToEmotion(dataset, basename)
        if emotion not in SELECTION_EMOTIONS: # On regarde si on s'intéresse à cette émotion
            continue
            
        features = extract_features(file, mfcc=True, chroma=True, mel=True) # Extraction des caractéristiques
        # Stockage du couple features/emotion
        X.append(features)
        y.append(emotion)

        # Séparation des données pour l'entraînement et le test
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [ ]:
"""
Chargement des datasets et des émotions sélectionnés
    - 75% des données pour l'entraînement
    - 25% pour le test 
"""
# Peut prendre plusieurs secondes/minutes selon la sélection de datasets/émotions 
X_train, X_test, y_train, y_test = load_data(test_size=0.25)


# Affichage du nombre d'audios utilisés :

print("[Pour l'entraînement] :", X_train.shape[0], "fichiers")
print("[Pour le test] :", X_test.shape[0], "fichiers")

print("[Pour chaque fichier audio] :", X_train.shape[1], "caractéristiques")

# Utilisation de plusieurs modèles

##  Avec DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
Arbre_decision = DecisionTreeClassifier(random_state=0, max_depth=20)
print('[*Entrainement du modèle*]')

# Entraînement
clf = Arbre_decision.fit(X_train, y_train)
# Prédiction
ypredit = clf.predict(X_test)
# Calcul de la précision
treeAccuracy = accuracy_score(y_true=y_test, y_pred=ypredit)

print("Précision pour l'arbre de décision: {:.2f}%".format(treeAccuracy*100))

## Avec SVM

In [ ]:
from sklearn import svm
clf = svm.SVC(gamma=0.001)
print('[*Entrainement du modèle*]')

#Entraînement
clf.fit(X_train,y_train)
# Prédiction
ypredit = clf.predict(X_test)
# Calcul de la précision
SVMAccuracy = accuracy_score(y_true=y_test, y_pred=ypredit)

print("Précision pour SVM: {:.2f}%".format(SVMAccuracy*100))

# Avec MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(max_iter=300)
print('[*Entrainement du modèle*]')

# Entraînement
clf.fit(X_train, y_train)
# Prédiction
ypredit = clf.predict(X_test)
# Calcul de la précision
MLPaccuracy = accuracy_score(y_true=y_test, y_pred=ypredit)

print("Précision pour MLPClassifier: {:.2f}%".format(MLPaccuracy*100))